In [4]:
import boto3
import time


def extract_text(bucket_name, file_name):
    textract = boto3.client('textract', region_name='ap-southeast-2')
    response = textract.start_document_text_detection(
        DocumentLocation={'S3Object': {'Bucket': bucket_name, 'Name': file_name}}
    )

    job_id = response['JobId']
#     print(f"Text detection job started with ID: {job_id}")

    status = "IN_PROGRESS"
    while status == "IN_PROGRESS":
#         print("Waiting for Textract to process the document...")
        time.sleep(1)  # Wait 1 seconds before polling again
        response = textract.get_document_text_detection(JobId=job_id)
        status = response['JobStatus']
#         print(f"Current Job Status: {status}")

    # Step 3: Extract and print lines of text
    text_lines = []
    if status == "SUCCEEDED":
        for block in response['Blocks']:
            if block['BlockType'] == 'LINE':  # Focus only on text lines
                text_lines.append(block['Text'])
    return "\n".join(text_lines)

In [5]:
bucket_name = "data15group3-resumes-pdf"
file_name = "HaydenSmith.pdf"
resume = extract_text(bucket_name, file_name)